This Notebook was made in Google Colab. So it don't have the virtual environment libraries, local files path and loggings.
Libraries has been installed externally and file paths are given by inputs.

In [1]:
!apt-get install inxi &> /dev/null
!inxi -C

CPU:
  Topology: Single Core model: Intel Xeon bits: 64 type: MT 
  L2 cache: 45.0 MiB 
  Speed: 2300 MHz min/max: N/A Core speeds (MHz): 1: 2300 2: 2300 


In [2]:
!lscpu |grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.30GHz


In [3]:
!nvidia-smi

Mon Jul 17 10:49:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# installing libraries
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 11.7 MB/s eta 0:00:00


In [5]:
!pip install python-box

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.0 MB/s eta 0:00:00


In [6]:
from box import ConfigBox
from box.exceptions import BoxValueError
from dataclasses import dataclass
from pathlib import Path
import yaml
import os

In [7]:
## helper functions


def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns
    Args:
        path_to_yaml (str): path like input
    Raises:
        ValueError: if yaml file is empty
        e: empty file
    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

In [8]:
@dataclass(frozen = True)
class TrainModelConfig:
  base_model: Path
  data_file: Path
  saved_model_dir: Path
  epochs: int
  batch_size: int
  image_size: int
  optimizer: str


In [9]:
CONFIG_FILE_PATH = "/content/drive/MyDrive/Object_Detection_Projects/config.yaml"
PARAMS_FILE_PATH = "/content/drive/MyDrive/Object_Detection_Projects/params.yaml"

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_train_model_config(self) -> TrainModelConfig:

      base_model_path = os.path.join(self.config.base_model.base_model_dir,
                                     self.params.BASE_MODEL.MODEL_NAME)
      config = self.config.train_model
      params = self.params.TRAIN_MODEL_PARAMS

      model_training_config = TrainModelConfig(
          base_model = Path(base_model_path),
          data_file = config.data_file_path,
          saved_model_dir = config.saved_model_dir,
          epochs = params.EPOCHS,
          batch_size= params.BATCH_SIZE,
          image_size = params.IMAGE_SIZE,
          optimizer = params.OPTIMIZER )

      return model_training_config

In [11]:
from ultralytics import YOLO
import torch

In [12]:
class TrainModel:
  def __init__(self, config: TrainModelConfig):
    self.config = config

  def train_model(self):
    try:
      model = YOLO(self.config.base_model)
      model.train(
          data = self.config.data_file,
          imgsz = self.config.image_size,
          epochs = self.config.epochs,
          batch = self.config.batch_size,
          project = self.config.saved_model_dir,
          optimizer= self.config.optimizer
      )
    except Exception as e:
      raise e


In [13]:
try:
  config = ConfigurationManager()
  train_model_config = config.get_train_model_config()
  model_training = TrainModel(train_model_config)
  model_training.train_model()

except Exception as e:
  raise e

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Object_Detection_Projects/yolov8l.pt, data=/content/drive/MyDrive/Object_Detection_Projects/data.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Object_Detection_Projects/train_models, name=None, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, a